# Example of IFU workflow based on WEAVE-APERTIF

This is a small example, akin to `TUTORIAL.md`. Please read that file (and `README`) for more details.

## Initialisation and dirty hacks

To import the different steps, we must add the `ifu` directory (the parent of the `workflow` directory where this Notebook is supposed to live) to the Python path.

*Disclaimer:* I have cheated in all but the very first step and used the shell commands in the tutorial rather than properly calling the Python functions. Even worse, I had to set the environment variable `PYTHONPATH` before every invocation, and even more worse than even worser, I just did it with the `export` command, assuming a `bash` Unix shell. I nominate my self for the *Quick and Dirty Hack of the Day Award* ;^D

In [36]:
import os
import sys
import numpy as np
from astropy.coordinates import SkyCoord, FK5, ICRS

if '..' not in sys.path:
    sys.path.append('..')
    print(sys.path)

We also specify the path to the `configure` tools (`configure` and `dither`) needed in stage 4.
I consider that hard-coding it this way is peanuts in comparison to the above ;^D

In [8]:
path_to_configure_tools = '../../configure_3.0.2/'

Set the location of the template files and the output directory (create the latter if necessary):

In [9]:
ifu_driver_template = os.path.join('aux', 'ifu_driver_template.fits')
fits_catalogue_template = os.path.join('aux', 'WA_CatalogueTemplate.fits')

output_dir = 'output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Stage 1: Creation of IFU driver catalogue (based on your input)

Fill the trimester data, as well as the e-mail addresses of the IP and SWG members:

In [10]:
survey = 'WA'
trimester = '2020A1'
PI_email = 'a@domain.com'
PI_first_name = 'Name'
PI_surname = 'Surname'
report_verbosity = 1
cc_report = 'b@domain.com,c@domain.com'

Fill the data of the desired targets. For this example, we chose 4 WA galaxies:
- HSB_HR is a high surface brightness galaxy, observed at high spectral resolution.
- HSB_LR is the same galaxy at low spectral resolution.
- LSB_LR is a low surface brightness galaxy at low spectral resolution.
- BWT_LR is a bad weather target at low spectral resolution.

In [40]:
data_dict = {}

data_dict['TARGSRVY'] = 3*[survey]
data_dict['TARGPROG'] = ['WA_HR', 'WA_LR', 'WA_LR']

data_dict['TARGID'] = ['HSB_HR', 'HSB_LR', 'ISB_LR']
data_dict['TARGNAME'] = ['PSZ1_nameHSB', 'PSZ1_nameHSB', 'PSZ1_nameISB']

data_dict['TARGPRIO'] = [10.0, 10.0, 10.0]

data_dict['PROGTEMP'] = ['61331', '41331', '41331']

data_dict['OBSTEMP'] = ['NBCEB', 'NBCEC', 'NBCEC']

data_dict['GAIA_RA'] = [48.14007743, 48.14007743, 22.94501069]
data_dict['GAIA_DEC'] = [39.31920434, 39.31920434, 38.71704016]

data_dict['GAIA_PMRA'] = [np.nan, np.nan, np.nan]
data_dict['GAIA_PMDEC'] = [np.nan, np.nan, np.nan]
data_dict['GAIA_PARAL'] = [np.nan, np.nan, np.nan]

data_dict['IFU_PA_REQUEST'] = [0.0, 0.0, 0.0]

data_dict['IFU_DITHER'] = [3, 3, 3]

Convert to ICRS, if necessary:

In [41]:
necessary = True  # coordinates are actually J2000
# necessary = False  # coordinates are already ICRS

if necessary:
    original_coords = SkyCoord(ra=data_dict['GAIA_RA'], dec=data_dict['GAIA_DEC'], unit='deg', frame=FK5(equinox='J2000'))
    coords_Gaia = original_coords.transform_to(ICRS())
    data_dict['GAIA_RA'] = coords_Gaia.ra.value
    data_dict['GAIA_DEC'] = coords_Gaia.dec.value

data_dict['GAIA_EPOCH'] = 3*['2015.5']

And that's it. No more input is needed from the user.

Simply

In [42]:
from workflow.stage1 import create_ifu_driver_cat

driver_catalog_filename = os.path.join(output_dir, '{}_{}-ifu_driver_cat.fits'.format(survey, trimester))

create_ifu_driver_cat(ifu_driver_template, data_dict, driver_catalog_filename,
                      trimester, PI_email, report_verbosity=report_verbosity,
                      cc_report=cc_report, overwrite=True)

'output/WA_2020A1-ifu_driver_cat.fits'

and check the results:

In [43]:
!export PYTHONPATH='..'; stage1/check_ifu_driver_cat.py $driver_catalog_filename

INFO:root:output/WA_2020A1-ifu_driver_cat.fits has passed the checks


In [44]:
!export PYTHONPATH='..'; stage1/plot_ifu_driver_cat.py $driver_catalog_filename

INFO:root:This plotting tool needs time. If you want to monitor its progress, see the directory which will contain its ouput images.


A few windows (one per target in `data_dict`) might pop up during the process. Check the images that are saved in the `output` directory.

## Stage 2: Creation of XML files for the targets in the catalog

*Warning*: By default, old XML files will not be overwritten. Instead, new files will be created, continuing the numbering!

This is usually not my intention, so I'll `rm output/*.xml`. Please skip or comment the following line if you would like to *append* the current driver catalogue to the existing target list.

In [45]:
xml_filename = os.path.join(output_dir, '{}_{}-*.xml'.format(survey, trimester))
!rm $xml_filename

In [46]:
!export PYTHONPATH='..'; stage2/create_xml_files.py $driver_catalog_filename

INFO:root:Reading output/WA_2020A1-ifu_driver_cat.fits
INFO:root:Processing 3 non-custom dither OBs for LIFU
INFO:root:	Writing to output/WA_2020A1-lifu_01-t.xml
INFO:root:	Writing to output/WA_2020A1-lifu_02-t.xml
INFO:root:	Writing to output/WA_2020A1-lifu_03-t.xml
INFO:root:Processing 0 custom dither OBs for LIFU
INFO:root:Processing 0 mIFU fields into 0 OBs according to all mode


## Stage 3: Adding guide and calibration stars to the XML files

In [47]:
xml_filename = os.path.join(output_dir, '{}_{}-*-t.xml'.format(survey, trimester))
!export PYTHONPATH='..'; stage3/add_guide_and_calib_stars.py $xml_filename

CRITICAL:root:Do not trust on the output of this code by now: This software is great, but the assumed position of the LIFU guide camera must be confirmed
INFO:root:2207 guide stars have been retrieved for (48.14006650564287, 39.319206770154004)
INFO:root:25 guide stars are useful for (48.14006650564287, 39.319206770154004)
INFO:root:1 guide stars has been selected for (48.14006650564287, 39.319206770154004) PA = 3.3848654938826996 deg
INFO:root:Notice that GAIA_DR value for the selected stars is GDR2
INFO:root:Requested value for PA has NOT been adopted
INFO:root:	Writing to output/WA_2020A1-lifu_01-tgc.xml
CRITICAL:root:Do not trust on the output of this code by now: This software is great, but the assumed position of the LIFU guide camera must be confirmed
INFO:root:2207 guide stars have been retrieved for (48.14006650564287, 39.319206770154004)
INFO:root:25 guide stars are useful for (48.14006650564287, 39.319206770154004)
INFO:root:1 guide stars has been selected for (48.1400665056

Now check the images `*tgc-guide_stars.png` in the `output` directory, which contain information about the guide stars and the final position angle adopted for the observation, which might not be the exactly the same as the requested PA.

## Stage 4: Configuring the XML files

Now it is time to run the configure scripts.

In [48]:
configure_script = os.path.join(path_to_configure_tools, 'lifu_configure')
dither_script = os.path.join(path_to_configure_tools, 'dither')

from glob import glob

xml_tgc_filename = os.path.join(output_dir, '{}_{}-*-tgc.xml'.format(survey, trimester))
for filename in glob(xml_tgc_filename):
    print(filename)
    tmp_filename = filename[:-4]+'s-tmp-orig.xml'
    new_filename = filename[:-4]+'s-orig.xml'
    lint_filename = filename[:-4]+'s.xml'
    !export PYTHONPATH='..'; $configure_script --epoch 2020 -i $filename -o $tmp_filename;
    !export PYTHONPATH='..'; $dither_script -i $tmp_filename -o $new_filename
    !xmllint --c14n $new_filename | xmllint --format --encode utf-8 --output $lint_filename -

output/WA_2020A1-lifu_01-tgc.xml
output/WA_2020A1-lifu_02-tgc.xml
output/WA_2020A1-lifu_03-tgc.xml


## Stage 5: Creation the IFU FITS catalogue

In [50]:
xml_tgcs_filename = os.path.join(output_dir, '{}_{}-*-tgcs.xml'.format(survey, trimester))
!export PYTHONPATH='..'; stage5/create_ifu_fits_cat.py $fits_catalogue_template $xml_tgcs_filename --cat_nme1 $PI_first_name --cat_nme2 $PI_surname --gaia_dr 2

## Stage 6: Improving the IFU FITS catalogue (optional, but recommended)

In [56]:
fits_catalogue_filename = os.path.join(output_dir, '{}_{}-ifu_from_xmls.fits'.format(survey, trimester))
!export PYTHONPATH='..'; stage6/add_PS1_photometry.py $fits_catalogue_filename

INFO:root:Downloading PS1 broadmand imaging for output/WA_2020A1-ifu_from_xmls.fits
INFO:root:HSB_HR: center=(48.1397468101801, 39.31896776337385); size=(0.15022935377384317 deg, 2163 pix)
INFO:root:HSB_HR_g.fits already exists
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja.
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.wcs.wcs]
this form of the PCi_ja keyword is deprecated, use PCi_ja.
stage6/add_PS1_photometry.py:172: RuntimeWarning: invalid value encountered in log10
  phot_table['aperture_sum'].data)
INFO:root:HSB_HR_r.fits already exists
this form of the PCi_ja keyword is deprecated, use PCi_ja. [astropy.w

The broadband images from PanSTARRS are saved in FITS format in a directory with the same name as the original "from_xml" catalog (one file per target per band), as well as PNG previews that illustrate the magnitudes per fibre that go into the final FITS catalogue.
This file is named as the original, stripping the "from_xml" part.

Now it's time for a formal check:

In [57]:
populated_fits_catalogue_filename = os.path.join(output_dir, '{}_{}-ifu.fits'.format(survey, trimester))
!export PYTHONPATH='..'; stage6/check_populated_ifu_fits_cat.py $fits_catalogue_filename $populated_fits_catalogue_filename

INFO:root:
INFO:root: fitsdiff: 4.0.2
INFO:root: a: output/WA_2020A1-ifu_from_xmls.fits
INFO:root: b: output/WA_2020A1-ifu.fits
INFO:root: HDU(s) not to be compared:
INFO:root:  SIM
INFO:root: Keyword(s) not to be compared:
INFO:root:  CHECKSUM DATASUM DATETIME MAG_G_CM MAG_I_CM MAG_R_CM
INFO:root: Table column(s) not to be compared:
INFO:root:  APS_FLAG APS_GAND_EBV APS_GAND_ERR APS_GAND_MODE APS_GAND_RED1
INFO:root:  APS_GAND_RED2 APS_IFU_MASK APS_IFU_SRC_DEC APS_IFU_SRC_ID
INFO:root:  APS_IFU_SRC_RA APS_IFU_TSSL_COVAR APS_IFU_TSSL_MIN_SNR
INFO:root:  APS_IFU_TSSL_TARG_SNR APS_IFU_TSSL_TYPE APS_LS_MODE APS_LS_NUM_MC
INFO:root:  APS_LS_RES APS_PPXF_DEG_ADD APS_PPXF_DEG_MULT APS_PPXF_MOM
INFO:root:  APS_PPXF_NUM_MC APS_PPXF_WL_MAX APS_PPXF_WL_MIN APS_SIGMA
INFO:root:  APS_SSP_NUM_CHAIN APS_SSP_NUM_WLKR APS_TEMPL_LIB APS_TEMPL_LIB_NORM
INFO:root:  APS_WL_MAX APS_WL_MIN APS_Z COORDCAT GAIA_MAG_BP GAIA_MAG_BP_ERR
INFO:root:  GAIA_MAG_G GAIA_MAG_G_ERR GAIA_MAG_RP GAIA_MAG_RP_ERR MAG_G
INFO

## Stage 7: Creation of the combo FITS catalogue

If you had a mixture of LIFU, mIFU, and MOS observations. We will ignore it for the time being.

In [60]:
# mos_filename = os.path.join(output_dir, '{}_{}-mos.fits'.format(survey, trimester))
final_catalogue_filename = populated_fits_catalogue_filename
# !export PYTHONPATH='..'; stage7/create_combo_fits_cat.py $mos_filename $populated_fits_catalogue_filename

## Stage 8: Submission to WASP of the combo FITS catalogue

This is the time to submit your populated FITS catalogue to the [WASP](http://wasp.ast.cam.ac.uk/) in order to get unique `CNAME` identifiers.

## Stage 9: Finishing the XML files

Now we go from FITS to XML format:

In [61]:
!export PYTHONPATH='..'; stage9/fill_xmls_with_fits_info.py $final_catalogue_filename $xml_tgcs_filename

INFO:root:Populating targets for file WA_2020A1-lifu_01.xml
INFO:root:	Writing to output/WA_2020A1-lifu_01.xml
INFO:root:Populating targets for file WA_2020A1-lifu_02.xml
INFO:root:	Writing to output/WA_2020A1-lifu_02.xml
INFO:root:Populating targets for file WA_2020A1-lifu_03.xml
INFO:root:	Writing to output/WA_2020A1-lifu_03.xml


## Stage 10: Submission of the final XML files to WASP

Just a final check:

In [65]:
final_xml_filename = os.path.join(output_dir, '{}_{}-lifu_[0-9][0-9].xml'.format(survey, trimester))
!export PYTHONPATH='..'; stage10/plot_data_from_xml.py $final_xml_filename

and, after verifying that everything seems to make sense, the files should be packed together into a single tar file and uploaded to the [WASP](http://wasp.ast.cam.ac.uk/).